# Problème 2 : Cryptographie

Un **chiffrement** est un procédé qui transforme un texte, le texte original, en un autre texte, le texte chiffré, de manière à rendre impossible (en principe) la reconstitution du texte original à partir du texte chiffré, sauf à connaître la **clé** de chiffrement, c'est-à-dire le ou les paramètres caractérisant l'opération de chiffrement. (Notons donc que *chiffrer* ne signifie pas *transformer en chiffres*.)

Dans ce problème, nous considérons que les textes à chiffrer sont écrits uniquement avec les 26 lettres de l'alphabet latin, en majuscule, sans [signes diacritiques](https://fr.wikipedia.org/wiki/Diacritiques_utilis%C3%A9s_en_fran%C3%A7ais), sans espace et sans ponctuation.

La **méthode de Hill** est une méthode de chiffrement qui consiste à découper le message en blocs de deux lettres et à remplacer chaque bloc de deux lettres par un autre bloc de deux lettres, selon une procédure impliquant un produit matrice-vecteur. La matrice utilisée dans ce produit (une matrice $2\times 2$ à coefficients dans $\{0,\ldots,25\}$) constitue la clé de chiffrement de la méthode.

Voyons sur un exemple comment fonctionne cette méthode. Nous prenons comme matrice de chiffrement la matrice

$$
M = \begin{pmatrix}2 & 19 \\ 5 & 12\end{pmatrix}.
$$

Le texte à chiffrer est $\texttt{MATHEMATIQUE}$.

**Etape 1.** Le texte doit d'abord être divisé en blocs de deux lettres.

$$\texttt{MA TH EM AT IQ UE}$$

Puis chaque bloc de deux lettres est converti en vecteur de deux nombres (A devenant 0, B devenant 1, C devenant 2, etc.)

$$(12, 0)\ (19, 7)\ (4, 12)\ (0, 19)\ (8, 16)\ (20, 4)$$

**Etape 2.** Chaque vecteur est transformé en le multipliant d'abord par la matrice $M$, puis, en prenant, pour chaque coefficient du vecteur résultat, le reste modulo $26$. 

Pour le vecteur $(12, 0)$, la multiplication par $M$ donne :

$$
\begin{pmatrix}2 & 19 \\ 5 & 12\end{pmatrix} \begin{pmatrix}12 \\ 0\end{pmatrix} = \begin{pmatrix}24 \\ 60\end{pmatrix}.
$$

Puis, en prenant le reste modulo $26$ des coefficients $24$ et $60$, on obtient $(24, 8)$.

En appliquant la même procédure à tous les vecteurs, on trouve :

$$
(24, 8)\ (15, 23)\ (2, 8)\ (23, 20)\ (8, 24)\ (12, 18)\ 
$$


**Etape 3.** Il reste à convertir les vecteurs de nombres en blocs de lettres (0 devenant A, 1 devenant B, 2 devenant C, etc.) Finalement, le texte chiffré est :

$$
\texttt{YIPXCIXUIYMS}
$$

**L'objectif de ce problème est de programmer le chiffrement de Hill, son déchiffrement et son attaque par force brute.**

## A. Rappels d'arithmétique

Nous considérons que les notions de **division euclidienne**, de **multiple**, de **diviseur** et de **PGCD** sont connues (voir le cours de L1 ou le document d'accompagnement sur l'arithmétique si nécessaire).

* On appelle **reste de $a$ modulo $b$** le reste de la division euclidienne de $a$ par $b$. On le note $a \operatorname{mod} b$. Quel opérateur Python permet de calculer ce reste ? A l'aide de cet opérateur, vérifier que $60 \bmod 26 = 8$, puis calculer $111 \operatorname{mod} 26$.

In [1]:
print(60 % 26)
print(111 % 26)

8
7


* On appelle **quotient entier de $a$ par $b$** le quotient de la division euclidienne de $a$ par $b$. Quel opérateur Python permet de calculer ce quotient ? A l'aide de cet opérateur, vérifier que le quotient entier de $60$ par $26$ est $2$, puis calculer le quotient entier de $111$ par $9$.

In [2]:
print(60 // 26)
print(111 // 9)

2
12


* Quelle fonction Python (du module `math`) permet de calculer le PGCD de deux entiers ? A l'aide de cette fonction, calculer le PGCD de 196 et 252, puis le PGCD de 196 et 225.

In [3]:
import math
print(math.gcd(196, 252))
print(math.gcd(196, 225))

28
1


On dit que deux entiers sont **premiers entre eux** si leur PGCD est égal à 1. (Attention : cette notion est distincte de la notion de nombre premier.)

* A l'aide d'un (court) programme Python, déterminer tous les nombres compris entre $1$ et $30$ qui sont premiers avec $32$ (vous devez obtenir $1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29$ ; même chose avec tous les nombres compris entre $1$ et $30$ qui sont premiers avec $10$.

In [4]:
print("nombres premiers avec 32:")
for i in range(1, 31):
    if math.gcd(i, 32) == 1:
        print(i)
        
print("\nnombres premiers avec 10:")
for j in range(1, 31):
    if math.gcd(j, 10) == 1:
        print(j)

nombres premiers avec 32:
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29

nombres premiers avec 10:
1
3
7
9
11
13
17
19
21
23
27
29


Soit $n\in \mathbb{N}^*$. On dit que deux entiers $a$ et $b$ sont **congrus modulo $n$** si $n$ divise $b-a$. On note cette relation $a \equiv b \pmod n$. Par exemple, $4 \equiv 10 \pmod 3$.

En particulier $a \equiv 0 \pmod n$ signifie que $n$ divise $a$.

Deux entiers $a$ et $b$ sont congrus modulo $n$ si et seulement s'il existe un entier $k$ tel que $a = b + kn$.

Deux entiers $a$ et $b$ sont congrus modulo $n$ si et seulement s'ils ont le même reste modulo $n$.

* Parmi les relations suivantes, lequelles sont vraies ?

    * $ 23 \equiv 9 \pmod 7$

    * $ 41 \equiv 21 \pmod 8$

    * $ -5 \equiv 7 \pmod 6$

    * $ -16 \equiv 16 \pmod{11}$

* $ 23 \equiv 9 \pmod 7$ est vraie car 23 = 9 + 2 * 7

* $ -5 \equiv 7 \pmod 6$ est vraie car -5 = 7 + (-2) * 6


La relation de congruence modulo $n$ est une [relation d'équivalence](https://fr.wikipedia.org/wiki/Relation_d%27%C3%A9quivalence). 

Par ailleurs, si  $a \equiv b \pmod n$ et  $c \equiv d \pmod n$, alors  $a +c \equiv b+d \pmod n$ et $ac \equiv bd \pmod n$.

## B. Chiffrement de Hill

### B.1. Matrices $2\times 2$

Nous choisissons de représenter informatiquement les vecteurs par des listes et les matrices par des listes de listes (chaque sous-liste contenant les coefficients d'une ligne).

Par exemple, le vecteur $(2, 1)$ est représenté par la liste `[2, 1]` et la matrice $\begin{pmatrix}1 & 0 \\ 2 & -3\end{pmatrix}$ est représentée par `[[1, 0], [2, -3]]`.

* Ecrire une fonction `mulmat` qui renvoie le produit de deux matrices $2\times 2$.

```py
>>> A = [[2, 1], [1, 3]]
>>> B = [[1, 2], [3 ,0]]
>>> mulmat(A, B)
[[5, 4], [10, 2]]
```

In [5]:
def mulmat(a, b):
    resultat = [[0] * len(a[0]) for i in range(len(a))]

    for ligne_a in range(len(a)):
        for col_b in range(len(b[0])):
            for ligne_b in range(len(b)):
                resultat[ligne_a][col_b] += int(a[ligne_a][ligne_b] * b[ligne_b][col_b]) 
                
            resultat[ligne_a][col_b] = resultat[ligne_a][col_b] % 26
    return resultat
            
A = [[2, 1], [1, 3]]
B = [[1, 2], [3 ,0]]

mulmat(A, B)

[[5, 4], [10, 2]]

* Ecrire une fonction `mulmatvec` qui calcule le produit d'une matrice $2\times 2$ et d'un vecteur de taille $2$.


```py
>>> A = [[2, 1], [1, 3]]
>>> v = [1, 2]
>>> mulmat(A, v)
[4, 7]
```

In [6]:
def mulmatvec(a, vect):
    resultat = [0, 0]
    
    for ligne_a in range(len(a)):
            for ligne_v in range(len(vect)):
                resultat[ligne_a] += (a[ligne_a][ligne_v] * vect[ligne_v])
                
            resultat[ligne_a] = resultat[ligne_a] % 26
                
    return resultat

A = [[2, 1], [1, 3]]
v = [1, 2]
mulmatvec(A, v)

[4, 7]

### B.2. Fonction de chiffrement

* Ecrire une fonction `cvn`qui transforme un caractère en nombre (`"A"` est transformé en `0`, `"B"` en `1`, etc.) et une fonctions `nvc`qui transforme un nombre en caractère (`0` est transformé en `"A"`, `1` en `"B"`, etc.).

```py
>>> cvn("P")
15
>>> nvc(16)
'Q'
```

Indication : on pourra utiliser les fonctions natives `chr` et `ord` (voir documentation Python).

In [7]:
def cvn(car):
        return ord(car) - 65
    
def nvc(entier):
        return chr(entier + 65) 

print(cvn("P"))
print(nvc(16))

15
Q


- Ecrire une fonction `chiffre` qui chiffre un texte par chiffrement de Hill. Elle prendra en paramètres `M`, la matrice de chiffrement, et `text`, le texte à chiffrer. Si le texte à chiffrer possède un nombre impair de caractères, on ajoutera un caractère final tiré aléatoirement (en utilisant la fonction `randint` du module `random` par exemple). 

In [8]:
import random

def chiffre(M, text):
    lst, lst_car, lst_code, lst_txt = [], [], [], []
    
    for car in text:
        lst_car.append(cvn(car))
                       
    if len(lst_car) % 2 != 0:
        lst_car.append(random.randint(0, 25))
        
    for j in range(0, len(lst_car), 2): 
        lst.append([lst_car[j], lst_car[j+1]])

    for vecteur in lst:
        v = mulmatvec(M, vecteur)
        lst_code.append([v[0]%26, v[1]%26])
        
    for a in lst_code:
        for b in a:
            lst_txt.append(nvc(b))

    return "".join(lst_txt)  

- Tester la fonction `chiffre` sur l'exemple de l'introduction.

In [9]:
chiffre([[2, 19],[5, 12]], "MATHEMATIQUE")

'YIPXCIXUIYMS'

### B.3. Toutes les matrices ne conviennent pas pour le chiffrement

* Chiffrer les blocs de lettres `VE` et `IR` avec la matrice $\begin{pmatrix}2 & 18 \\ 6 & 12\end{pmatrix}$.

In [10]:
print(chiffre([[2, 18],[6, 12]], "VE"))
print(chiffre([[2, 18],[6, 12]], "IR"))

KS
KS


Nous observons qu'avec certaines matrices de chiffrement, deux blocs de lettres différents sont transformés en un même bloc de lettres. Autrement dit, la procédure de chiffrement n'est pas **injective** (et donc pas **bijective** non plus).

Dans ce cas, il est impossible de déchiffrer, c'est-à-dire de retrouver le texte original à partir du texte chiffré et de la matrice de chiffrement.

## C. Déchiffrement

Dans cette partie, nous cherchons à savoir quelles sont les matrices de chiffrement admissibles et comment procéder au déchiffrement.

Pour cela, nous allons d'abord formaliser mathématiquement la procédure de chiffrement de Hill.

On note $E$ l'ensemble des couples de nombres compris entre $0$ et $25$. La méthode de Hill associée à la matrice de chiffrement $M$ consiste en fait à utiliser l'application $\varphi : E \rightarrow E$ définie par

$$\varphi : \begin{pmatrix}
x \\ y
\end{pmatrix} \mapsto M \begin{pmatrix}
x \\ y
\end{pmatrix} \quad \operatorname{mod} 26,$$

où la notation $\operatorname{mod} 26$ signifie qu'on prend le reste modulo $26$ du vecteur $M \begin{pmatrix}
x \\ y
\end{pmatrix}$.

Nous pouvons alors présenter la méthode de chiffrement de Hill ainsi.

* **Etape 1.** Conversion des blocs de lettres en vecteurs de nombres.
* **Etape 2.** Application de la fonction $\varphi$ sur les vecteurs.
* **Etape 3.** Conversion des vecteurs de nombres en blocs de lettres.

Les matrices de chiffrement admissibles sont donc celles qui donnent une fonction $\varphi$ bijective.

Dans ce cas, nous pouvons procéder au déchiffrement de la façon suivante. 

* **Etape 1.** Conversion des blocs de lettres en vecteurs de nombres.
* **Etape 2.** Application de la fonction réciproque $\varphi^{-1}$ sur les vecteurs.
* **Etape 3.** Conversion des vecteurs de nombres en blocs de lettres.


### C.1. Inverse modulaire d'un entier

Soit $n \in \mathbb{N}^*$. Soit $a \in \mathbb{Z}$. On dit que $b \in \mathbb{Z}$ est un **inverse de $a$ pour la multiplication modulo $n$** (ou plus simplement un **inverse de $a$ modulo $n$**) si :
$$
a b \equiv 1 \pmod n.
$$

Si un nombre admet un inverse modulo $n$, on dit qu'il est **inversible** modulo $n$.

Par exemple, $8$ est inversible modulo $11$ et $7$ est un inverse de $8$ car $7 \times 8 \equiv 56 \equiv 1 \pmod{11}$. Les entiers $18$ et $-4$ sont aussi des inverses de $8$ modulo $7$.

Attention : contrairement aux nombres réels, l'inverse n'existe pas toujours. Par exemple $2$ n'a pas d'inverse modulo $6$.

Voici quelques résultats fondamentaux sur les inverse modulaires (que nous démontrerons dans la partie E).
   * Un entier $a$ est inversible modulo $n$ si et seulement si $a$ et $n$ sont premiers entre eux.
   * Si $a$ est inversible modulo $n$ alors, il existe un unique inverse compris entre $1$ et $n-1$.

Pour trouver un inverse de $a$ modulo $n$, nous proposons l'algorithme suivant.

*On parcourt les nombres de $1$ à $n-1$ et on teste chaque nombre (en calculant son produit avec $a$ modulo $n$) pour voir s'il s'agit d'un inverse modulo $n$. On renvoie le premier inverse trouvé. Si l'on n'en trouve aucun, on renvoie `None`.*

* Ecrire une fonction `invmod` qui prend en paramètre un entier $a$ et un entier $n$, et qui met en œuvre l'algorithme de calcul d'inverse décrit ci-dessus. Tester cette fonction sur les exemples suivants. 
```py
>>> invmod(8, 11)
7
>>> invmod(2, 6)
None
```

In [11]:
def invmod(a, n):
    if math.gcd(a, n) != 1:
        return None
    
    for i in range(1, n):
        if (i * a % n) == 1:
            return i 
    
print(invmod(8, 11))
print(invmod(2, 6))

7
None


### C.2. Inverse modulaire d'une matrice d'entiers

On dit que deux vecteurs (ou deux matrices) sont congrus modulo $n$ si tous leurs coefficients sont congrus modulo $n$ deux à deux. On utilise la même notation que pour les entiers.

Par exemple,

$$
\begin{pmatrix}
7 & 3 \\ 10 & 1
\end{pmatrix}
\equiv
\begin{pmatrix}
2 & 3 \\ 0 & 11
\end{pmatrix}
\pmod 5
$$

Soit $n \in \mathbb{N}^*$. Soit $A$ une matrice $2\times 2$ à coefficients entiers. On dit qu'une matrice $2\times 2$ à coefficients entiers $B$ est une **matrice inverse de $A$ pour la multiplication modulo $n$** (ou plus simplement un **matrice inverse de $A$ modulo $n$**) si :

$$
A B \equiv \begin{pmatrix}
1 & 0 \\ 0 & 1
\end{pmatrix} \pmod n.
$$

Si une matrice admet une matrice inverse modulo $n$, on dit qu'elle est **inversible** modulo $n$.

On définit le **déterminant** d'une matrice $\begin{pmatrix}
a & b \\ c & d
\end{pmatrix}$ comme la quantité $ad-bc$.

* Soit $M = \begin{pmatrix}
a & b \\ c & d
\end{pmatrix}$. On suppose que son déterminant est inversible modulo $26$ et on note $\delta$ l'inverse compris entre $0$ et $25$ de ce déterminant.
Vérifier que la matrice

$$
B = \delta \begin{pmatrix}
d & -b \\ -c & a
\end{pmatrix}
$$

est une matrice inverse de $A$.

$M = \begin{pmatrix}
a & b \\ c & d
\end{pmatrix}$ 

On a alors $det M = ad - bc$     avec $ad - bc \ne 0$

$\delta = \frac{1}{det M} = \frac{1}{ad - bc}$

Vérifions que B est la matrice inverse de A, c'est à dire: $B = \delta \begin{pmatrix} d & -b \\ -c & a \end{pmatrix} = A^{-1}$

Soit $A = \begin{pmatrix}a & b \\ c & d\end{pmatrix}$

Pour cela, on veut vérifier que $AB \equiv \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix} \pmod {26} $

$AB = \begin{pmatrix}a & b \\ c & d\end{pmatrix} \times  \delta \begin{pmatrix} d & -b \\ -c & a \end{pmatrix}$

$AB = \delta \begin{pmatrix}a & b \\ c & d\end{pmatrix} \times  \begin{pmatrix} d & -b \\ -c & a \end{pmatrix}$

$AB = \delta \begin{pmatrix}ab - cb & -ab + ab \\ cd - cd & ab - cb\end{pmatrix}$

$AB = \delta \begin{pmatrix}ab - cb & 0 \\ 0 & ab - cb\end{pmatrix}$

$AB = \frac{1}{ad - bc}  \times \begin{pmatrix}ab - cb & 0 \\ 0 & ab - cb\end{pmatrix}$

$AB = \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix}$

Donc $AB \equiv \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix} \pmod {26} $, B est bien une matrice inverse de A.

Nous avons ainsi montré qu'une matrice était inversible modulo $n$ si son déterminant était inversible modulo $n$. Nous admettons que la réciproque est vraie.

Nous admettons aussi que si $B$ est une matrice inverse de $A$ modulo $n$, la matrice $B \operatorname{mod} n$ (c'est-à-dire la matrice dont les coefficients sont les restes modulo $n$ des coefficients de $B$) est aussi une matrice inverse de $A$ modulo $n$. Cette matrice inverse $B \operatorname{mod} n$ est une matrice dont tous les coefficients sont compris entre $0$ et $n-1$. C'est la seule matrice inverse possédant cette propriété.

*  Ecrire une fonction `det` qui calcule le déterminant d'une matrice $2 \times 2$.

In [12]:
def det(M):
    a, b = M[0]
    c, d = M[1]
    return a*d - b*c

print(invmod(det([[24, 8], [15, 23]]), 26))

None


* Ecrire une fonction `invmatmod` qui prend en paramètres une matrice `M` (de taille $2 \times 2$) et un entier `n`, et qui renvoie la matrice inverse modulo $n$ dont tous les coefficients sont compris entre $0$ et $n-1$ (ou `None` si la matrice n'est pas inversible modulo $n$). 

In [13]:
def invmatmod(M, n):
    determinant = invmod(det(M), n)
    if determinant == None:
        return None
    
    M_inverse = [[(M[1][1] * determinant) % n, (-M[0][1] * determinant) % n], [(-M[1][0] * determinant) % n, (M[0][0] * determinant) % n]]
    rest = mulmat(M, M_inverse)
    
    if rest == [[1, 0], [0, 1]]:
        return M_inverse
    else:
        return None

* Vérifier, grâce à `invmatmod`, que l'inverse modulo $26$ de $\begin{pmatrix} 2 & 1 \\ 1 & 3 \end{pmatrix}$ est $\begin{pmatrix} 11 & 5 \\ 5 & 16 \end{pmatrix}$.

In [14]:
invmatmod([[2, 1], [1, 3]], 26)

[[11, 5], [5, 16]]

* Quel est l'inverse modulo $26$ de $\begin{pmatrix} 2 & 1 \\ 5 & 12 \end{pmatrix}$ ?  

In [15]:
invmatmod([[2, 1], [5, 12]], 26)

[[2, 15], [23, 22]]

### C.3. Fonction de déchiffrement de Hill

Il n'est pas difficile de vérifier que l'application $\varphi : E \rightarrow E$ définie par :

$$
\varphi : \begin{pmatrix}
x \\ y
\end{pmatrix} \mapsto M \begin{pmatrix}
x \\ y
\end{pmatrix} \quad \operatorname{mod} 26
$$

est bijective si et seulement si la matrice $M$ est inversible modulo $26$.

De plus, l'application réciproque $\varphi^{-1}$ est alors 
$$
\varphi^{-1} : \begin{pmatrix}
x \\ y
\end{pmatrix} \mapsto W \begin{pmatrix}
x \\ y
\end{pmatrix} \quad \operatorname{mod} 26
$$

où $W$ est une matrice inverse modulo $26$ de $M$.

- Ecrire une fonction `dechiffre` qui déchiffre un text chiffré par chiffrement de Hill. Elle prendra en paramètres `M`, la matrice de chiffrement, et `text`, le text à déchiffrer.

In [16]:
def dechiffre(M, text):
    lst, lst_car, lst_code, lst_txt = [], [], [], []
    M = invmatmod(M, 26)
    
    for car in text: 
        lst_car.append(cvn(car))
        
    if len(lst_car) % 2 != 0:
        lst_car.append(random.randint(0, 25))
    
    for j in range(0, len(lst_car), 2):  
        lst.append([lst_car[j], lst_car[j+1]])

    for c in lst:
        chiffre = mulmatvec(M,c)
        lst_code.append([chiffre[0]%26,chiffre[1]%26])

    for a in lst_code:
        for b in a:
            lst_txt.append(nvc(b))
            
    return "".join(lst_txt)     

- En utilisant la fonction `dechiffre`, déchiffrer le texte `TXWYIZATBFCUOBXZKSBFGAT` sachant qu'il a été chiffré avec la matrice $\begin{pmatrix} 1 & 2 \\ 1 & 3 \end{pmatrix}$.

In [17]:
print(dechiffre([[1, 2],[1, 3]], "TXWYIZATBFCUOBXZKSBFGAT"))

LESCAROTTESSONTCUITESUPP


## D. Attaque par force brute d'un chiffrement de Hill

* Combien y a-t-il de matrices $2\times 2$, à coefficients dans $\{0,\ldots,25\}$ ?

Les coefficients sont dans $\{0,\ldots,25\}$, donc 26 possibilités pour chaques coefficients de la matrice.
Sachant qu'une matrice $2\times 2$ possède 4 coefficients, on a alors $26^4 = 456 976$ .

Il y a donc $456 976$ matrices à coefficients dans $\{0,\ldots,25\}$.

* A l'aide d'un programme, déterminer le nombre de matrices $2\times 2$, à coefficients dans $\{0,\ldots,25\}$, inversibles modulo $26$.

In [18]:
def nb_matrice():
    cmpt = 0
    for a in range(26):
        for b in range(26):
            for c in range(26):
                for d in range(26): 
                    if invmatmod([[a, b], [c, d]], 26) != None:
                        cmpt += 1
    return cmpt

print(nb_matrice())

157248


Le nombre de matrices permet d'envisager une attaque par force brute pour décrypter un texte chiffré par un chiffrement dont on ne possède pas la matrice.

*On parcourt toutes les matrices de chiffrement possibles. Pour chaque matrice, on déchiffre le texte, puis on teste s'il est intelligible (dans une langue donnée). Pour cela, on compte le nombre de mots de la langue présents dans le texte.*

Sur la page du cours, on trouvera un fichier `dicofr.txt` contenant une liste de référence de mots de la langue française.

- Lire et stocker dans une liste le contenu du fichier `dicofr.txt`.

In [19]:
fichier = open("dicofr.txt", "r")
lst = fichier.readlines()
fichier.close()
print(lst)

['ABANDON\n', 'ABANDONNES\n', 'ABBAS\n', 'ABRI\n', 'ABUS\n', 'ACCUSE\n', 'ACHETAIT\n', 'ACTE\n', 'ADJOINT\n', 'ADOLPHE\n', 'ADORATION\n', 'ADORE\n', 'AFFINITES\n', 'AFFREUX\n', 'AFRIQUE\n', 'AGIR\n', 'AIME\n', 'AIR\n', 'ALFRED\n', 'ALLAIS\n', 'ALLUMONS\n', 'AMBITIEUX\n', 'AME\n', 'AMES\n', 'AMI\n', 'AMIE\n', 'AMOUR\n', 'ANCETRE\n', 'ANCIENNE\n', 'ANESTHESIE\n', 'ANGLAIS\n', 'ANGLETERRE\n', 'ANGOULEME\n', 'ANONYME\n', 'ANONYMES\n', 'ANTIQUE\n', 'APHASIE\n', 'APHONSE\n', 'APPELLE\n', 'APPRENTI\n', 'APRES\n', 'ARMEES\n', 'ATTAQUE\n', 'AUDIET\n', 'AUDOUIN\n', 'AUGUSTE\n', 'AURES\n', 'AURIOL\n', 'AUSSI\n', 'AUTRE\n', 'AUTRES\n', 'AUX\n', 'AVAIS\n', 'AVANCEMENT\n', 'AVEC\n', 'AVERTISSEMENT\n', 'AVOIR\n', 'ABAILARD\n', 'ABANDONNEZ\n', 'ABASOURDI\n', 'ABATTONS\n', 'ABBADIE\n', 'ABBAS\n', 'ABBE\n', 'ABDERAME\n', 'ABIAD\n', 'ABOMINATION\n', 'ABORDER\n', 'ABRAHAM\n', 'ABRUZZE\n', 'ABSOLUMENT\n', 'ABSORBE\n', 'ABSORBES\n', 'ABSTENEZ\n', 'ABYSSINIE\n', 'ABYSSINIENS\n', 'ACADEMIE\n', 'ACCABLE\n', 'A

- Décrypter le message `MHWGCMZXWEFJIIRFESXEVNWMWNFRWGMHZMZLQMBNTNKNBBJZRMKQEFNQBLMHCLCVRKVV` sachant qu'il s'agit d'un message écrit en français et chiffré avec un chiffrement de Hill. (Remarque : l'exécution du programme d'attaque peut prendre un certain temps.)

In [ ]:
def decryptage():
    for a in range(26):
        for b in range(26):
            for c in range(26):
                for d in range(26): 

                    if invmatmod([[a, b], [c, d]], 26) != None:

                        text = dechiffre([[a, b],[c, d]], "MHWGCMZXWEFJIIRFESXEVNWMWNFRWGMHZMZLQMBNTNKNBBJZRMKQEFNQBLMHCLCVRKVV")

                        for e in range(len(lst)):
                            result = text.find(lst[e])

                            if result >= 0:
                                print("lst[e]:", lst[e], "result:", result)
                                return lst[e]
    return None

decryptage()
                        